# Implement the Recursive Algorithm of Decision Tree

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def value_counts(data):
    # TODO Calculate the number of each label.
    # Return a dict with the labels as keys, and
    # their accurances as values
    
    d = np.unique(data[ : , -1], return_counts=True)
    return dict(zip(d[0], d[1]))

def divide_data(data, feature_column, feature_val):
    data1 = []
    data2 = []
    # TODO split the data into two parts by feature_column,
    # where data1 contains all with value at feature column less than
    # feature_value, and data2 contains all values larger that feature_val 
    
    data1 = data[data[ : , feature_column] > feature_val]
    data2 = data[data[ : , feature_column] <= feature_val]
    return data1, data2

def gini(data):
    #TODO calculate the gini
    
    dict_ = value_counts(data)
    print(dict_)
    sum_ = 0
    for value in dict_.values():
        sum_ += (value / data.shape[0]) ** 2
    gini = 1 - sum_
    return gini

def entropy(data):
    #TODO calculate the entropy
    
    dict_ = value_counts(data)
    entropy = 0
    for value in dict_.values():
        entropy += -((value / data.shape[0]) * np.log2(value / data.shape[0]))
    return entropy

def square_loss(data):
    #TODO calculate the loss
    
    y_mean = np.mean(data[ : , -1])
    loss = np.mean((y_mean - data[ : , -1]) ** 2) / 2
    return loss

class DecisionNode(object):
    def __init__(self,
                 column = None,
                 value = None,
                 false_branch = None,
                 true_branch = None,
                 current_results = None,
                 is_leaf = False):
        """
        column: column is the index of feature by wich data is splitted
        value: value is column's value by which we filter data into splits
        true_branch: boolean, if True, it is the true branch of it's parent
        false_branch: boolean, if True, it is the false branch of it's parent
        is_leaf: boolean, if True, node has no child
        current_results: is value_counts(data) for data which reached this node
        """
         
        self.column = column
        self.value = value
        self.false_branch = false_branch
        self.true_branch = true_branch
        self.current_results = current_results
        self.is_leaf = is_leaf

def build_tree(data, current_depth = 0, max_depth = 4, criterion = "gini", task = "classification"):
    """
    Task can be classification or regression
    Criterion is inpurity function to use
    """

    if len(data) == 0:
        return DecisionNode(is_leaf = True)

    if current_depth == max_depth:
        return DecisionNode(current_results = value_counts(data), is_leaf = True)
    
    if len(value_counts(data)) == 1:
        return DecisionNode(current_results = value_counts(data), is_leaf = True)

    # TODO, calculate best split 
    # split_pos = []
    # split_neg = []
    #search best_criterion, best_column, best_value
    best_criterion = gini(data)
    for column in range(data.shape[1]):
        for value in data[ : , column]:
            data1, data2 = divide_data(data, feature_column = column, feature_val = value)     
            information_gain = data1.shape[0] / (data1.shape[0] + data2.shape[0]) * gini(data1) + data2.shape[0] / (data1.shape[0] + data2.shape[0]) * gini(data2)
            if information_gain < best_criterion:
                best_criterion = information_gain
                best_value = value
                best_column = column  
     
    #getting split_pos and split_neg
    split_pos, split_neg = divide_data(data, best_column, best_value)
    
    # if we cannot improve by splitting:
    if information_gain == best_criterion:
        return DecisionNode(current_results= value_counts(data), is_leaf = True)
    else:
        return DecisionNode(column = best_column,
                            value = best_value,
                            current_results = value_counts(data),
                            false_branch = build_tree(split_neg, current_depth + 1, max_depth),
                            true_branch = build_tree(split_pos, current_depth + 1, max_depth))


In [3]:
class DecisionTree(object):
    def __init__(self, max_tree_depth = 4, criterion = "gini", task = "classification"):
        self.max_depth = max_tree_depth
        self.tree = None
        self.task = task
        
        self.criterion = gini
        if criterion == "entropy":
            self.criterion = entropy
        if criterion == "square_loss":
            self.criterion = square_loss

    def fit(self, X, Y):
        # build data
        data = np.concatenate((X, Y), axis = 1)  #building data
        self.tree = build_tree(data,
                               task = self.task,
                               max_depth = self.max_depth, 
                               criterion = self.criterion)
    def predict(self, X):
        Y = []
        # TODO get labels       
        return Y

# Perform Results on Iris Dataset

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()

X = iris.data
Y = iris.target.reshape(iris.target.shape[0], 1)
data = np.concatenate((X, Y), axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [5]:
model = DecisionTree()
model.fit(X_train, Y_train)

{0.0: 34, 1.0: 32, 2.0: 39}
{0.0: 16, 1.0: 30, 2.0: 38}
{0.0: 18, 1.0: 2, 2.0: 1}
{1.0: 6, 2.0: 19}
{0.0: 34, 1.0: 26, 2.0: 20}
{1.0: 2, 2.0: 14}
{0.0: 34, 1.0: 30, 2.0: 25}
{1.0: 12, 2.0: 32}
{0.0: 34, 1.0: 20, 2.0: 7}
{1.0: 2, 2.0: 14}
{0.0: 34, 1.0: 30, 2.0: 25}
{0.0: 2, 1.0: 21, 2.0: 38}
{0.0: 32, 1.0: 11, 2.0: 1}
{2.0: 1}
{0.0: 34, 1.0: 32, 2.0: 38}
{1.0: 7, 2.0: 26}
{0.0: 34, 1.0: 25, 2.0: 13}
{0.0: 2, 1.0: 25, 2.0: 38}
{0.0: 32, 1.0: 7, 2.0: 1}
{1.0: 7, 2.0: 26}
{0.0: 34, 1.0: 25, 2.0: 13}
{1.0: 7, 2.0: 26}
{0.0: 34, 1.0: 25, 2.0: 13}
{0.0: 20, 1.0: 31, 2.0: 38}
{0.0: 14, 1.0: 1, 2.0: 1}
{1.0: 7, 2.0: 26}
{0.0: 34, 1.0: 25, 2.0: 13}
{2.0: 11}
{0.0: 34, 1.0: 32, 2.0: 28}
{1.0: 6, 2.0: 19}
{0.0: 34, 1.0: 26, 2.0: 20}
{1.0: 12, 2.0: 32}
{0.0: 34, 1.0: 20, 2.0: 7}
{0.0: 23, 1.0: 32, 2.0: 39}
{0.0: 11}
{1.0: 14, 2.0: 35}
{0.0: 34, 1.0: 18, 2.0: 4}
{0.0: 2, 1.0: 21, 2.0: 38}
{0.0: 32, 1.0: 11, 2.0: 1}
{0.0: 2, 1.0: 21, 2.0: 38}
{0.0: 32, 1.0: 11, 2.0: 1}
{0.0: 2, 1.0: 25, 2.0: 38}
{0.

{2.0: 23}
{0.0: 34, 1.0: 32, 2.0: 16}
{0.0: 9, 1.0: 32, 2.0: 39}
{0.0: 25}
{2.0: 2}
{0.0: 34, 1.0: 32, 2.0: 37}
{1.0: 11, 2.0: 39}
{0.0: 34, 1.0: 21}
{1.0: 22, 2.0: 39}
{0.0: 34, 1.0: 10}
{1.0: 27, 2.0: 39}
{0.0: 34, 1.0: 5}
{2.0: 4}
{0.0: 34, 1.0: 32, 2.0: 35}
{2.0: 4}
{0.0: 34, 1.0: 32, 2.0: 35}
{1.0: 4, 2.0: 37}
{0.0: 34, 1.0: 28, 2.0: 2}
{1.0: 1, 2.0: 35}
{0.0: 34, 1.0: 31, 2.0: 4}
{0.0: 9, 1.0: 32, 2.0: 39}
{0.0: 25}
{1.0: 27, 2.0: 39}
{0.0: 34, 1.0: 5}
{2.0: 2}
{0.0: 34, 1.0: 32, 2.0: 37}
{2.0: 23}
{0.0: 34, 1.0: 32, 2.0: 16}
{0.0: 9, 1.0: 32, 2.0: 39}
{0.0: 25}
{1.0: 11, 2.0: 39}
{0.0: 34, 1.0: 21}
{1.0: 11, 2.0: 39}
{0.0: 34, 1.0: 21}
{2.0: 28}
{0.0: 34, 1.0: 32, 2.0: 11}
{1.0: 11, 2.0: 39}
{0.0: 34, 1.0: 21}
{0.0: 9, 1.0: 32, 2.0: 39}
{0.0: 25}
{0.0: 1, 1.0: 32, 2.0: 39}
{0.0: 33}
{0.0: 30, 1.0: 32, 2.0: 39}
{0.0: 4}
{2.0: 28}
{0.0: 34, 1.0: 32, 2.0: 11}
{1.0: 27, 2.0: 39}
{0.0: 34, 1.0: 5}
{2.0: 11}
{0.0: 34, 1.0: 32, 2.0: 28}
{0.0: 9, 1.0: 32, 2.0: 39}
{0.0: 25}
{2.0: 39}
{0

{0.0: 9, 1.0: 32}
{0.0: 25}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{}
{0.0: 34, 1.0: 32}
{}
{0.0: 34, 1.0: 32}
{1.0: 32}
{0.0: 34}
{1.0: 32}
{0.0: 34}
{1.0: 32}


In [6]:
from sklearn.datasets import load_boston
boston = load_boston() 
